# Proceso de ETL previo a la construccion de la plataforma API

### Importamos las librerías.

In [39]:
import pandas as pd 
import numpy as pd
import ast 
import warnings
import json
warnings.filterwarnings('ignore')

In [40]:
%xmode minimal

Exception reporting mode: Minimal


Leemos el arhivo original. No siendo usar *pandas* directamente, usamos el módulo *ast* de python.

In [41]:
# Cargar el archivo JSON linea por linea

archivo_json = 'output_steam_games.json'
rows = []

with open( archivo_json ) as f:
    for line in f:
        rows.append(json.loads(line))


Convertimos los datos en un dataframe de pandas, y ejecutamos *info* para tener una mejor idea de los tipos de datos interpretados por pandas.

In [42]:
import pandas as pd
df = pd.DataFrame(rows)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


Para una observación más cercana, leemos la cabeza y la cola del dataset, a la vez que observamos el diccionario de datos provisto.

In [14]:
df.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.tail()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"
120444,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550,NaN


Llaman mi atención, registros que no tienen datos, los observo más de cerca.

In [7]:
# Cuántos registros sin datos hay en el dataset
rows_with_13_null_columns = df[df.isnull().sum(axis=1).isin([13])]
print(rows_with_13_null_columns.head())
dataset_without_null = int(df.shape[0]) - int(rows_with_13_null_columns.shape[0])
print('Cantidad de registros con algun dato en alguna columna:' ,dataset_without_null)


Empty DataFrame
Columns: [publisher, genres, app_name, title, url, release_date, tags, reviews_url, specs, price, early_access, id, developer]
Index: []
Cantidad de registros con algun dato en alguna columna: 32135


La cantidad de datos es significativo, paso a eliminarlos.

In [43]:
df = df.dropna(how='all')
df.shape

(32135, 13)

Verifico si hay URLs de Steam duplicadas (ya que cada una es indicativa de un juego único, independientemente del id, título o nombre de la aplicación)

In [44]:
has_duplicates = df['url'].duplicated()
total_duplicates = has_duplicates.sum()
print(total_duplicates)

0


Eliminamos las columnas de id, URL, reviews URL. title y app_name serán exploradas más tarde

In [45]:
df.drop('id', axis=1, inplace=True)
df.drop('url', axis=1, inplace=True)
df.drop('reviews_url', axis=1, inplace=True)

Armo un diccionario para convertir tipos de dato a string

In [46]:
data_types_dict = {'publisher': 'string',
 'app_name': 'string',
 'price': 'string',
 'developer': 'string',
 'sentiment': 'string',#
 'metascore': 'string'}#

Ejecuto

In [ ]:
df = df.astype(data_types_dict)

Convierto la columna metascore a float

In [ ]:
df['metascore'] = pd.to_numeric(df['metascore'], errors='coerce')

Fuerzo la conversión de la columna release_date como tipo datetime para poder investigar los datos usando ese campo

In [47]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

Visualizo cuántos registros por año hay

In [48]:
release_year_count = df.groupby(df['release_date'].dt.year).size()
release_year_dict = release_year_count.to_dict()
release_year_dict

{1970.0: 2,
 1975.0: 1,
 1980.0: 1,
 1981.0: 3,
 1982.0: 3,
 1983.0: 4,
 1984.0: 4,
 1985.0: 3,
 1986.0: 1,
 1987.0: 9,
 1988.0: 7,
 1989.0: 10,
 1990.0: 12,
 1991.0: 16,
 1992.0: 15,
 1993.0: 34,
 1994.0: 36,
 1995.0: 33,
 1996.0: 39,
 1997.0: 42,
 1998.0: 61,
 1999.0: 55,
 2000.0: 40,
 2001.0: 69,
 2002.0: 50,
 2003.0: 84,
 2004.0: 79,
 2005.0: 102,
 2006.0: 155,
 2007.0: 167,
 2008.0: 213,
 2009.0: 341,
 2010.0: 442,
 2011.0: 590,
 2012.0: 1193,
 2013.0: 1552,
 2014.0: 2844,
 2015.0: 4952,
 2016.0: 6942,
 2017.0: 9560,
 2018.0: 123,
 2019.0: 4,
 2021.0: 1}

Considerando el archivo json de origen data del año 2018, decido eliminar del dataset los registros de 2019 y 2021, por no estar lanzados aún

In [49]:
a_eliminar = [2019, 2021]
df = df.query('release_date.dt.year not in @a_eliminar')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32130 entries, 88310 to 120444
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   publisher     24078 non-null  object        
 1   genres        28847 non-null  object        
 2   app_name      32128 non-null  object        
 3   title         30080 non-null  object        
 4   release_date  29889 non-null  datetime64[ns]
 5   tags          31967 non-null  object        
 6   specs         31460 non-null  object        
 7   price         30756 non-null  object        
 8   early_access  32130 non-null  object        
 9   developer     28831 non-null  object        
dtypes: datetime64[ns](1), object(9)
memory usage: 2.7+ MB


Viendo las similitudes de las columnas app_name y title en el dicionario de datos y en los datos observados, corroboro cuántos son una coincidencia exacta

In [50]:
coincidencias = df['app_name'] == df['title']
cantidad_coincidencias = coincidencias.sum()
cantidad_coincidencias

29525

Observo que hay un valor ausente en app_name

In [51]:
app_name_invalido = df[df['app_name'].isnull()]
print(app_name_invalido)

      publisher           genres app_name title release_date             tags  \
88384       NaN              NaN      NaN   NaN          NaT              NaN   
90890       NaN  [Action, Indie]      NaN   NaN   2014-08-26  [Action, Indie]   

                            specs  price early_access developer  
88384                         NaN  19.99        False       NaN  
90890  [Single-player, Game demo]    NaN        False       NaN  


Ese valor ausente no puede ser subsanado copiando un valor de la columna title. Ya estamos listos para eliminar title.

In [52]:
df.drop('title', axis=1, inplace= True)

Para las primeras 6 funciones de la FAST API app (o más tarde en el modelo) no vamos a necesitar discount_price. Eliminamos la columna

In [ ]:
df.drop('discount_price', axis=1, inplace=True)

Por similitud, dejamos genre, eliminamos tags.

In [54]:
df.drop('tags', inplace= True, axis=1)

Exploramos la columna price, obteniendo valores únicos diferentes a NA

In [55]:
precios = df['price'].dropna().unique()

Armo una lista para almacenar valores de esta lista que no puedan convertirse a floats

In [56]:
no_floats = []

for valor in precios:
    try:
        float(valor)
    except ValueError:
        no_floats.append(valor)

Observo qué valores puedo equiparar a cero para el precio

In [57]:
no_floats

['Free To Play',
 'Free to Play',
 'Free',
 'Free Demo',
 'Play for Free!',
 'Install Now',
 'Play WARMACHINE: Tactics Demo',
 'Free Mod',
 'Install Theme',
 'Third-party',
 'Play Now',
 'Free HITMAN™ Holiday Pack',
 'Play the Demo',
 'Starting at $499.00',
 'Starting at $449.00',
 'Free to Try',
 'Free Movie',
 'Free to Use']

Elijo cuáles equivalen a precio cero

In [58]:
free = ['Free To Play',
 'Free to Play',
 'Free',
 'Play for Free!',
 'Free to Use']

Elijo los que serán NA

In [59]:
na = ['Free Demo',
 'Install Now',
 'Play WARMACHINE: Tactics Demo',
 'Free Mod',
 'Install Theme',
 'Third-party',
 'Play Now',
 'Free HITMAN™ Holiday Pack',
 'Play the Demo',
 'Starting at $499.00',
 'Starting at $449.00',
 'Free to Try',
 'Free Movie']

Observo si algo similar puede intuirse de la columna genres

In [60]:
df_exploded = df.explode('genres')
unique_genres = df_exploded['genres'].dropna().unique()
unique_genres

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing',
       'Early Access', 'Massively Multiplayer',
       'Animation &amp; Modeling', 'Video Production', 'Utilities',
       'Web Publishing', 'Education', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting'], dtype=object)

En efecto, 'Free to Play' esta presente. Lo uso para sustituir el precio en la columna price, usando 0.

In [61]:
mask = df['genres'].apply(lambda genre_list: isinstance(genre_list, list) and 'Free to Play' in genre_list)

df.loc[mask, 'price'] = '0'

Ahora sustituyo los precios con cero en todas las filas donde los valores sean uno de los que recopilé en la lista free

In [62]:
df['price'] = df['price'].replace(free, '0')

Convierto a NA los que están en la lista na

In [63]:
df['price'] = df['price'].replace(na, pd.NA)

Finalmente, convierto a float

In [ ]:
df['price'] = df['price'].astype(float)

Exploro la columna sentiments

In [ ]:
unique_sentiments = df['sentiment'].unique().dropna().tolist()
print(unique_sentiments)

Hay varios valores que no expresan sentiments, sólo cantidad de reviews. Los reemplazo con NA.

In [ ]:
values_to_remove = [sentiment for sentiment in unique_sentiments if 'review' in sentiment]

In [ ]:
df['sentiment'] = df['sentiment'].replace(values_to_remove, pd.NA)

In [ ]:
unique_sentiments = df['sentiment'].unique().tolist()
print(unique_sentiments)

Reinicio el index

In [ ]:
df.reset_index(drop=True, inplace=True)

Última vista de la info

In [ ]:
df.info()

Defino el nombre del archivo

In [65]:
parquet_file = 'steam_data_clean.parquet'

Guardo el dataset como archivo de parquet para posterior uso de la plataforma FAST API, para sus primeras 6 funciones

In [66]:
df.to_parquet(parquet_file, engine="pyarrow", use_deprecated_int96_timestamps=True)

ImportError: Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.

In [68]:
import pandas as pd

# Suponiendo que 'df' es tu DataFrame
parquet_file = "C:/Users/steff/OneDrive/Documentos/Segundo_proyecto_individual/nueva_base.parquet"
df.to_parquet(parquet_file, engine="pyarrow", use_deprecated_int96_timestamps=True)


ImportError: Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.